# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.dataset import Dataset
from azureml.core import Environment

# deployment
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
import json, requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-model'
project_folder = './iris-classification-project-3'
experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-model,quick-starts-ws-153057,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
# get compute target
amlcompute_name = 'notebook153057'

compute_target = ComputeTarget(workspace=ws, name=amlcompute_name)
compute_target.wait_for_completion(show_output=True)


Running


In [4]:
#get data from kaggle for titanic csv
example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
dataset = Dataset.Tabular.from_delimited_files(example_data, header=False)        
#Register Dataset in Workspace
dataset = dataset.register(workspace=ws,
                                   name="Iris Classification",
                                   description="Classification problem for iris species")


df = dataset.to_pandas_dataframe()
df.head()

,Column1,Column2,Column3,Column4,Column5
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
df["Column5"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":20,
    "max_concurrent_iterations":4,
    "primary_metric":'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                            compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Column5",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
    )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-model,AutoML_caf261ae-59c9-46ec-8f48-a0d74b5ddd7d,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
remote_run.wait_for_completion()

{'runId': 'AutoML_caf261ae-59c9-46ec-8f48-a0d74b5ddd7d',
 'target': 'notebook153057',
 'status': 'Completed',
 'startTimeUtc': '2021-07-30T21:16:24.851073Z',
 'endTimeUtc': '2021-07-30T21:57:31.402879Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'notebook153057',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-model","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group":"aml-quickstarts-153057","workspace_name":"quick-starts-ws-153057","region":"southcentralus","compute_target":"notebook153057","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
#get the best model and display
best_model, automl = remote_run.get_output()
print(best_model)
print(automl)

Package:azureml-automl-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-core, training version:1.32.0, current version:1.31.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-defaults, training version:1.32.0, current version:1.31.0
Package:azureml-interpret, training version:1.32.0, current version:1.31.0
Package:azureml-mlflow, training version:1.32.0, current version:1.31.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.31.0
Package:azureml-telemetry, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-runtime, training version:1.32.0, current version:1.31.0


Run(Experiment: automl-model,
Id: AutoML_caf261ae-59c9-46ec-8f48-a0d74b5ddd7d_3,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                                      class_weight=None, criterion='entropy',
                                      max_depth=None, max_features='log2',
                                      max_leaf_nodes=None, max_samples=None,
                                      min_impurity_decrease=0.0,
                                      min_impurity_split=None,
                                      min_samples_leaf=0.01,
                                      min_samples_split=0.103684

In [16]:
#TODO: Save the best model
registered_model = remote_run.register_model(model_name="best_amlmodel_iris")
registered_model.download(target_dir="./output/aml_model")

'output/aml_model/model.pkl'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
# create ACI webservice config for deployment
container_instance_config= AciWebservice.deploy_configuration(
    description="Automl model deployment to Azure container instance",
    cpu_cores = 1,
    memory_gb=4,
    enable_app_insights=True
)

aml_model = Model(ws, "best_amlmodel_iris")

In [18]:
# get env for Inference config
env = Environment.get(workspace=ws, name="AzureML-AutoML")

In [19]:
# Inference config and final deployment

inference_config = InferenceConfig(entry_script="score.py", environment=env)

service = Model.deploy(workspace=ws, 
                       name='best-amlmodel-iris', 
                       models=[aml_model], 
                       inference_config=inference_config, 
                       deployment_config=container_instance_config)

In [20]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-30 22:16:35+00:00 Creating Container Registry if not exists.
2021-07-30 22:16:35+00:00 Registering the environment.
2021-07-30 22:16:37+00:00 Use the existing image.
2021-07-30 22:16:37+00:00 Generating deployment configuration.
2021-07-30 22:16:38+00:00 Submitting deployment to compute.
2021-07-30 22:16:41+00:00 Checking the status of deployment best-amlmodel-iris..
2021-07-30 22:20:50+00:00 Checking the status of inference endpoint best-amlmodel-iris.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [21]:
# data to test the endpoint after deployment
data = {
    "data":
    [
        { 
            "Column1":2.3,
            "Column2":3.4,
            "Column3":2.9,
            "Column4":5.6
        }
    ]
    
}

print("URI to which we will send requests", service.scoring_uri)

# Convert to JSON string
input_data = json.dumps(data)
print(input_data)
# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(service.scoring_uri, input_data, headers=headers)
print(resp.text)

URI to which we will send requests http://58a355a5-a4af-4e15-96d7-242488d229de.southcentralus.azurecontainer.io/score
{"data": [{"Column1": 2.3, "Column2": 3.4, "Column3": 2.9, "Column4": 5.6}]}
["Iris-virginica"]


TODO: In the cell below, print the logs of the web service and delete the service

In [22]:
print(service.get_logs())

2021-07-30T22:20:41,012471100+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-07-30T22:20:41,021881900+00:00 - iot-server/run 
2021-07-30T22:20:41,021899700+00:00 - rsyslog/run 
2021-07-30T22:20:41,050599500+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_2381ca41b26ae71cb9e5e93b8c3d0e43/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-07-30T22:20:41,380560000+00:00 - iot-server/finish 1 0
2021-07-30T22:20:41,382310800+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (60)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 88
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-07-30 22:20:46,560 | root | INFO | Starting up app insights client
logging socket was 

In [23]:
service.delete()

In [ ]:
compute_target.delete()